In [ ]:
import utility

utility.lj_potential(0.5,10)

In [ ]:
utility.pbc(43,10)

In [ ]:
import init
#parameters
rho = 0.01
Npart = 108
box_length = (Npart / rho) ** (1/3)
# print(box_length)
bond_length = 2.353  #angstrom

k = 30
temp = 300 #kelvin
beta = 1

debug = True
accepted_steps = 0

In [ ]:
positions = init.init_system(box_length, Npart)


In [ ]:
positions
import random
import numpy as np


In [ ]:
idx = random.randint(0,Npart-1)
idx

In [ ]:
prev_position = np.copy(positions[idx])
prev_position

In [ ]:
Wo1 = k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
Wo1

In [ ]:
Wo2 = np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
for i in range(k-1):
    r = utility.generate_random_unit_vector() * bond_length
    positions[idx][1] = positions[idx][0] + r
    Wo2 += np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
    
Wo = Wo1 * Wo2
print(Wo)

In [ ]:
positions[idx][0] = np.random.rand(3)* box_length
positions[idx][0]

In [ ]:
Wn1 =  k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
Wn1

In [ ]:
#select a random_chain
    global accepted_steps
    idx = random.randint(0,Npart-1)
    #find rosenbluth factor for previous configuration
    prev_position = np.copy(positions[idx])
    Wo1 = k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
    Wo2 = np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
    for i in range(k-1):
        r = utility.generate_random_unit_vector() * bond_length
        positions[idx][1] = positions[idx][0] + r
        Wo2 += np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
    
    Wo = Wo1 * Wo2
    #find rosenbluth factor for new configuration
    #choose a random postion for first atom
    positions[idx][0] = np.random.rand(3)* box_length
    Wn1 =  k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
    second_atom_pos = []
    Wn2 = []
    for i in range(k):
        r = utility.generate_random_unit_vector()* bond_length
        positions[idx][1] = positions[idx][0] + r
        second_atom_pos.append(positions[idx][1])
        Wn2.append(np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length)))

    Wn2_sum = sum(Wn2)

    #select a configuration i with probaility Wn2[i]/Wn2_sum
    cum_Wn2 = Wn2[0]
    r_Wn2_sum = random.random()*Wn2_sum
    i = 0
    while cum_Wn2 < r_Wn2_sum:
        i +=1
        cum_Wn2 += Wn2[i]
    print(i, Wn2[i])
    Wn = Wn1 * Wn2_sum
    #replace position of second atom with selected second atom configuration 
    positions[idx][1] = second_atom_pos[i]
    print(Wo, " ", Wn)
    if Wn < Wo and random.random() > Wn/Wo:
        #not accept
        positions[idx] = prev_position   
    else :
        accepted_steps += 1
        if debug:
            print("new move accepted")
    
    return 


In [ ]:
import init
import utility
import visualize
import random
import numpy as np

#parameters
rho = 0.01
Npart = 108
box_length = (Npart / rho) ** (1/3)
# print(box_length)
bond_length = 2.353  #angstrom

k = 30
temp = 300 #kelvin
beta = 1

debug = True
accepted_steps = 0

positions = init.init_system(box_length, Npart)

def check(positions):
    for pos in positions:
        if np.linalg.norm(pos[0] - pos[1]) > bond_length + 0.01:
            return False
    
    return True

def get_minimum_distance(positions, idx1, idx2):
    min_len = 1000
    for i in range(len(positions)):
        if idx1 !=i:
            for pos in positions[i]:
                min_len = min(min_len, utility.pbc(np.linalg.norm(pos - positions[idx1][idx2]),box_length))
    
    return min_len

def CBMC_step(positions, Npart):
    #select a random_chain
    global accepted_steps
    idx = random.randint(0,Npart-1)
    #find rosenbluth factor for previous configuration
    prev_position = np.copy(positions[idx])
    Wo1 = k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
    Wo2 = np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
    for i in range(k-1):
        r = utility.generate_random_unit_vector() * bond_length
        positions[idx][1] = positions[idx][0] + r
        Wo2 += np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
    # print("Wo2" ,Wo2)
    Wo = Wo1 * Wo2
    #find rosenbluth factor for new configuration
    #choose a random postion for first atom
    positions[idx][0] = np.random.rand(3)* box_length
    Wn1 =  k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
    second_atom_pos = []
    Wn2 = []
    for i in range(k):
        r = utility.generate_random_unit_vector()* bond_length
        positions[idx][1] = positions[idx][0] + r
        second_atom_pos.append(positions[idx][1])
        Wn2.append(np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length)))
        # if get_minimum_distance(positions,idx,1) <1 and np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length)) >1:
            # print(Wn2[i],np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length)) )

    Wn2_sum = sum(Wn2)
    # print(Wn2_sum)
    #select a configuration i with probaility Wn2[i]/Wn2_sum
    cum_Wn2 = Wn2[0]
    r_Wn2_sum = random.random()*Wn2_sum
    i = 0
    while cum_Wn2 < r_Wn2_sum:
        i +=1
        cum_Wn2 += Wn2[i]
    # print(i, Wn2[i])
    Wn = Wn1 * Wn2_sum
    #replace position of second atom with selected second atom configuration 
    positions[idx][1] = second_atom_pos[i]
    # print(Wo, " ", Wn)
    if Wn < Wo and random.random() > Wn/Wo:
        #not accept
        positions[idx] = prev_position   
    else :
        accepted_steps += 1
        if debug:
            print("new move accepted")
            print(Wn2[i], Wn1,Wn2[i]/Wn2_sum, get_minimum_distance(positions,idx,0),get_minimum_distance(positions,idx,1))
            if get_minimum_distance(positions,idx,1) < 1:
                print(np.exp(-1*utility.energy_of_particle(idx,1,positions,box_length)))
                # print(Wn2)
                
                # return True
    
    return False

total_energy_sum = 0

for i in range(100):
    # visualize.visualize(positions)
    energy = utility.total_energy(positions,box_length)
    # print(energy)
    total_energy_sum += energy
    if CBMC_step(positions,Npart) == True:
        break


print(f"avg_energy {total_energy_sum / 100}")
print(f"Acceptance percentage {accepted_steps/1}")
visualize.visualize(positions)


In [ ]:
a = [5,3,2,1,4]
b = sum(a)
b
a[1]

In [ ]:

global accepted_steps
idx = random.randint(0,Npart-1)
#find rosenbluth factor for previous configuration
prev_position = np.copy(positions[idx])
Wo1 = k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
Wo2 = np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
for i in range(k-1):
    r = utility.generate_random_unit_vector() * bond_length
    positions[idx][1] = positions[idx][0] + r
    Wo2 += np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length))
# print("Wo2" ,Wo2)
Wo = Wo1 * Wo2
#find rosenbluth factor for new configuration
#choose a random postion for first atom
positions[idx][0] = np.random.rand(3)* box_length
Wn1 =  k * np.exp(-beta * utility.energy_of_particle(idx,0,positions,box_length))
second_atom_pos = []
Wn2 = []
for i in range(k):
    r = utility.generate_random_unit_vector()* bond_length
    positions[idx][1] = positions[idx][0] + r
    second_atom_pos.append(positions[idx][1])
    Wn2.append(np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length)))
    # if get_minimum_distance(positions,idx,1) <1 and np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length)) >1:
        # print(Wn2[i],np.exp(-beta * utility.energy_of_particle(idx,1,positions,box_length)) )

Wn2_sum = sum(Wn2)
# print(Wn2_sum)
#select a configuration i with probaility Wn2[i]/Wn2_sum
cum_Wn2 = Wn2[0]
r_Wn2_sum = random.random()*Wn2_sum
i = 0
while cum_Wn2 < r_Wn2_sum:
    i +=1
    cum_Wn2 += Wn2[i]
# print(i, Wn2[i])
Wn = Wn1 * Wn2_sum
#replace position of second atom with selected second atom configuration 
positions[idx][1] = second_atom_pos[i]
# print(Wo, " ", Wn)
if Wn < Wo and random.random() > Wn/Wo:
    #not accept
    positions[idx] = prev_position   
else :
    accepted_steps += 1
    if debug:
        print("new move accepted")
        print(Wn2[i], Wn1,Wn2[i]/Wn2_sum, get_minimum_distance(positions,idx,0),get_minimum_distance(positions,idx,1))
        if get_minimum_distance(positions,idx,1) < 1:
            print(np.exp(-1*utility.energy_of_particle(idx,1,positions,box_length)))
            # print(Wn2)
            
            # return True

In [ ]:
a= np.array((3,1))
c = []
for i in range(10):
    a +=1
    c.append(a)
print(c)

In [ ]:
import visualize

visualize.plot_energy_from_file("energy.txt")

In [ ]:
import matplotlib.pyplot as plt
with open("energy.txt",'r') as f:
    energy = [float(line.strip()) for line in f]
    steps = [ int(100 * i) for i in range(len(energy))]
    plt.plot(steps, energy)
    plt.ylabel("Energy (KbT)")
    plt.xlabel("Number of steps")
    plt.grid()
    plt.show()

In [ ]:
np_positions = np.load('positions.npy', allow_pickle=True)


In [ ]:
positions = []
for mol_pos in np_positions:
    positions.append([mol_pos[0],mol_pos[1]])

In [ ]:
utility.total_energy(positions,box_length)

In [ ]:
import init
from config import *
import utility
positions = init.init_system(box_length,Npart)

In [ ]:
len(positions)

In [ ]:
import visualize, utility
visualize.visualize(positions)
utility.total_energy(positions,box_length)

In [ ]:
from ethane_cbmc import *
# from tempfile import TemporaryFile
# position_file = TemporaryFile()

# positions = init.init_system(box_length, Npart)
# positions = utility.read_positions_from_file("positions.npy")

# energy_file = open("energy.txt",'a' )

for i in range(nsteps):
    # visualize.visualize(positions)
    energy = utility.total_energy(positions,box_length)
    print(f"Energy of the system at step {i} is: {energy}")
    # if i%write_interval == 0:
        # energy_file.write(str(energy)+"\n")
    positions = CBMC_step(positions,Npart)

# np.save('positions.npy', np.array(positions, dtype=object), allow_pickle=True)


In [ ]:
positions_list = [init.init_system(box_length,Npart),init.init_system(box_length,Npart)]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

def animate(i):
    # fig = plt.figure()
    ax.cla()
    positions = positions_list[i]
    for pos in positions:
        ax.plot([pos[0][0],pos[1][0]],[pos[0][1],pos[1][1]],
        [pos[0][2],pos[1][2]], 'g')
        ax.scatter3D([pos[0][0]],[pos[0][1]], [pos[0][2]], color = 'red')
        ax.scatter3D([pos[1][0]],[pos[1][1]], [pos[1][2]], color = 'blue')
    

num_plots = (len(positions_list))
fig = plt.figure(figsize=(20,20))
ax = plt.axes(projection='3d')
ani = FuncAnimation(fig, animate, frames=num_plots, interval=200)

writer = PillowWriter(fps=1)
ani.save('plots.gif', writer=writer)


In [ ]:
import init
import visualize
import utility
import numpy as np
with open('nvt.gro','r') as f:
    lines = [line for line in f]
    positions = []
    for i in range(Npart):
        l1 = lines[2*i+2].split()
        l2 = lines[2*i+3].split()
        mol_pos_1 = np.array([l1[3],l1[4],l1[5]]).astype(float)
        mol_pos_2 = np.array([l2[3],l2[4],l2[5]]).astype(float)
        positions.append([mol_pos_1,mol_pos_2])

positions

In [ ]:
lines

In [ ]:
from config import *
visualize.visualize(positions)

In [ ]:
utility.total_energy(positions,2.2)

In [ ]:
import init
import visualize
import utility
import numpy as np
positions = utility.read_positions_from_gromacs_file('nvt.gro')

In [ ]:
positions

In [ ]:
dx, dy, dz = positions[0][0] - positions[0][1]

In [ ]:
dx

In [ ]:
positions[0][0]

In [ ]:
positions[0][1]

In [ ]:
from config import *
utility.total_energy(positions,box_length)

In [ ]:
import init
from utility import generate_random_unit_vector
import numpy as np
from config import *
import random
import utility


In [ ]:
positions = init.init_system(box_length,108)

In [ ]:
import visualize
visualize.visualize_propane(positions)

In [ ]:
len(positions)

In [ ]:
for mol_pos in positions:
    r1 = mol_pos[2] - mol_pos[1]
    r2 = mol_pos[0] - mol_pos[1]
    r1 = utility.convert_to_unit_vector(r1)
    r2 = utility.convert_to_unit_vector(r2)
    print(np.arccos(np.dot(r1,r2)))

In [ ]:
# r1,r2 = init.generate_random_molecule(1.5)
r1 = np.array([0,0,0])
r2 = np.array([1.5,0,0])

In [ ]:
np.pi

In [ ]:
mol_pos = [r1,r2]

In [ ]:
def add_bond(mol_pos, beta = 1):
    r = mol_pos[-2] - mol_pos[-1]
    r = r / np.linalg.norm(r)
    print(r)
    flag = False
    while flag == False:
        r_new = generate_random_unit_vector()
        theta = np.arccos(np.dot(r,r_new))
        
        energy = 0.5*k_prop*(theta- theta_mean)**2
        if theta > 1.5:
            print(theta)
            print("energy" ,energy)
        
        if random.random() < np.exp(-beta*energy):
            r_new  = r_new *  bond_length + mol_pos[-1]
            flag = True
            return r_new
    


In [ ]:
r3 = add_bond(mol_pos)

In [ ]:
mol_pos.append(r3)


In [ ]:
mol_pos

In [ ]:
positions = [mol_pos]
import visualize
visualize.visualize_propane(positions)

In [ ]:
from utility import *
def add_bond_optimal(mol_pos, beta = 1):
    z = mol_pos[-2] - mol_pos[-1]
    y = np.array([0,1,0]) 
    if z[1] != 0:
        y = np.array([1,-(z[0]+z[2])/z[1],1])

    x = np.cross(y,z)

    #convert to unit vectors
    z = convert_to_unit_vector(z)
    y = convert_to_unit_vector(y)
    x = convert_to_unit_vector(x)

    #adding bond in spherical coordintate system for ease 

    r = bond_length
    theta = theta_mean
    phi = random.random() * (6.28)
    print(theta_mean)
    #converting this back to cartesian coordintates

    return mol_pos[-1] + (r*np.sin(theta)*np.cos(phi))*x + (r*np.sin(theta)*np.sin(phi))*y + (r*np.cos(theta))*z



In [ ]:
r1 = np.array([0,0,0])
r2 = np.array([1,0,0])

mol_pos = [r1,r2]

In [ ]:
r3 = add_bond_optimal(mol_pos)


In [ ]:
print(np.arccos(np.dot(convert_to_unit_vector(r3-r2),convert_to_unit_vector(r1-r2))))

In [ ]:
r3

In [ ]:
np.dot(r3-r2,r1-r2)

In [ ]:
import utility
positions = utility.read_positions_from_file("../project_runs/run10/8700/positions.npy")

In [ ]:
positions

In [ ]:
import visualize
visualize.visualize_propane(positions)

In [ ]:
from config import *
utility.total_energy(positions,box_length)

In [ ]:
import numpy as np
energy = np.zeros(Npart)
for i in range(len(positions)):
    for j in range(i+1,len(positions)):
        e = utility.find_potential_between_alkanes(positions[i],positions[j],box_length)
        energy[i] += e
        energy[j] +=e



In [ ]:
utility.energy_of_particle(77,0,positions,box_length)

In [1]:
import init
import utility
import visualize
import numpy as np
import random

#parameters

from config import *

def MC_step(positions, Npart):
    
    idx = random.randint(0,Npart-1)
    Uo = utility.energy_of_a_chain(idx, positions, box_length)
    prev_position = np.copy(positions[idx])
    positions[idx] = init.generate_random_molecule(bond_length)
    Un = utility.energy_of_a_chain(idx,positions,bond_length)
    print(Un, Uo)
    if random.random() < np.exp(-beta *(Un-Uo)):
        print(f"New state accepted")
        return positions, Un-Uo
    else :
        positions[idx] = prev_position
        print(f"Old state accepted")
        return positions, 0

    

positions = init.init_system(box_length, Npart)
energy = utility.total_energy(positions,box_length)



356976833.6148417 -0.4489337185285087
Old state accepted
0
energies 122.80466431259822 122.80466431259822
-0.2140264573530293 -0.8270809200499379
New state accepted
0.6130544626969087
energies 123.41771877529513 5770.420513612467
23.15858274242435 -1.4903633797035378
Old state accepted
0
energies 123.41771877529513 5770.420513612467
-0.4289408368715096 5646.78876837983
New state accepted
-5647.217709216701


/tmp/ipykernel_4948/1955306976.py:19: RuntimeWarning: overflow encountered in exp
  if random.random() < np.exp(-beta *(Un-Uo)):


energies -5523.799990441406 123.34305517579531
68745.58909088289 -2.0516575808888926
Old state accepted
0
energies -5523.799990441406 123.34305517579531
669.5353373426424 -0.4662899292062043
Old state accepted
0
energies -5523.799990441406 123.34305517579531
-1.6403602959814645 -0.5061907597280597
New state accepted
-1.1341695362534048
energies -5524.93415997766 123.50062471683466
-0.9953088199491463 42.27802858705267
New state accepted
-43.27333740700181
energies -5568.207497384662 80.98638337646364
245903.12662319795 1.1875534399527055
Old state accepted
0
energies -5568.207497384662 80.98638337646364
-0.12136140179581562 -1.811876444989592
Old state accepted
0
energies -5568.207497384662 80.98638337646364
10818896.28399732 -1.429687582972529
Old state accepted
0
energies -5568.207497384662 80.98638337646364
144966030841.7312 -0.18440365745278792
Old state accepted
0
energies -5568.207497384662 80.98638337646364
126.28326645805289 -1.7579661885271427
Old state accepted
0
energies -55

KeyboardInterrupt: 